<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# Advertools - Bulk status code checker with response headers

**Tags:** #advertools #adviz #website #analyze #audit #seo #status_code #response_headers

**Author:** [Elias Dabbas](https://www.linkedin.com/in/eliasdabbas/)

**Description:** This notebook runs an automated status code checker with response headers using the HTTP `HEAD` method.

* Bulk and concurrent checking of status codes for a known list of URLs
* Get all available response headers from all URLs
* Set speed, number of concurent request and various other crawling options
* Does NOT download the full HTML of a page, saving a lot of time, energy, and resources, and enabling an extreemely fast and light process

**References:**
- [advertools `crawl_headers` function](https://advertools.readthedocs.io/en/master/advertools.header_spider.html)
- Scrapy [throttling](https://docs.scrapy.org/en/latest/topics/autothrottle.html) and [custom settings](https://docs.scrapy.org/en/latest/topics/settings.html)

## Input

### Import libraries

In [ ]:
try:
    import advertools as adv
    import adviz
    import pandas as pd
    import plotly.express as px
    pd.options.display.max_columns = None
except ModuleNotFoundError:
    !pip install adviz

### Setup Variables

**Function parameters:**
- `url_list`: List of URLs to check the status codes
- `output_file`: The path to the file for saving the output. It has to be in the `.jl` format. Note that new lines to the file are `appended` to the end and the file is not overwritten while crawling.

**Custom settings:**

- `CONCURRENT_REQUESTS_PER_DOMAIN`: Defaults to 8. Most likely you will need to slow it down, because the crawling is extremely fast.
- `DEFAULT_REQUEST_HEADERS`: A dictionary where you can set custom request headers.
- `USER_AGENT`: Set a different/custom user agent if you need to.
- `AUTOTHROTTLE_ENABLED`: Whether or not to dynamically change the pace of crawling to be nice to servers as much as possible. It's usually good to enable this extension, as this type of crawling is extremely fast, often leading to 429 (too many requests code), or being blocked.

### Create helper visualizations

In [ ]:
def ecdf(df, column, template='none', width=None, height=500):
    df = df.sort_values(column)
    df['count below'] = range(len(df))
    df['count above'] = range(len(df)-1, -1, -1)
    df['total count'] = len(df)
    fig = px.ecdf(
        df.dropna(subset=[column]),
        x=column,
        markers=True, lines=False,
        template=template,
        height=height,
        width=width,
        hover_data=['count below', 'count above', 'total count'],
        title=f"Cumulative distribution of <b>{column.replace('_', ' ')}</b>",
        ecdfnorm='percent')
    fig.data[0].hovertemplate = f'<b>{column.replace("_", " ").title()}</b><br><br>{column}' + ': %{x}<br>percent: %{y}<br><br>count below: %{customdata[0]:,}<br>count above: %{customdata[1]:,}<br>total count: %{customdata[2]:,}<extra></extra>'
    fig.layout.yaxis.ticksuffix = '%'
    fig.layout.yaxis.showspikes = True
    fig.layout.xaxis.showspikes = True
    return fig

### Convert the website's XML sitemap to a `DataFrame`

Optionally get any other list of URLs hrough crawling, or any analytics tool.

In [1]:
sitemap = adv.sitemap_to_df('https://www.maven.com/robots.txt')

In [ ]:
url_list = sitemap['loc']  # or another list of URLs

output_file = 'crawl_headers_output.jl'

custom_settings = {
    # optionally change request headers:
    # 'DEFAULT_REQUEST_HEADERS': {},
    'AUTOTHROTTLE_ENABLED': True,
    'CONCURRENT_REQUESTS_PER_DOMAIN': 4,
    'USER_AGENT': adv.spider.user_agent,
    'LOG_FILE': 'crawl_headers_output.log',
}
custom_settings

{'AUTOTHROTTLE_ENABLED': True,
 'CONCURRENT_REQUESTS_PER_DOMAIN': 4,
 'USER_AGENT': 'advertools/0.13.2',
 'LOG_FILE': 'crawl_headers_output.log'}

## Model

### Check status codes and retreive response headers for `url_list`

In [ ]:
adv.crawl_headers(
    url_list=url_list,
    output_file=output_file,
    custom_settings=custom_settings
)

Open the crawl output file:

In [2]:
headers_df = pd.read_json('crawl_headers_output.jl', lines=True)
headers_df

#### Visualize status codes

In [3]:
adviz.status_codes(headers_df['status'])

In [4]:
adviz.value_counts_plus(headers_df['status'], name='Status codes', size=14)

#### Visualize URL structure

In [5]:
adviz.url_structure(
    headers_df['url'],
    domain='maven.com',
    items_per_level=50,
    theme='plotly_dark')

#### Get redirects

In [6]:
if 'redirect_urls' in headers_df:
    redirect_df = headers_df.filter(regex='^url$|redirect_').dropna(thresh=4)
    redirect_df['redirect_urls'] = redirect_df['redirect_urls'].str.split('@@')
    redirect_df['redirect_reasons'] = redirect_df['redirect_reasons'].astype(int).astype(str).str.split('@@')
    redirect_df['redirect_chain'] = (redirect_df['redirect_urls']
                                     .str.join('@@')
                                     .add('@@')
                                     .add(redirect_df['url']).str.split('@@'))
    from IPython.display import display
    display(redirect_df)
else:
    print('No redirects found in this dataset')

### Visualize download latency
The same charts can be used for any other `float` columns in `headers_df` if available.

In [7]:
fig = px.histogram(
    x=headers_df['download_latency'],
    template='none',
    labels={'x': 'Latency (seconds)<br>'},
    height=500,
    # experiment with different values for nbins:
    # nbins=10,
    title='<b>Download latency distribution</b>')

fig.add_annotation(
    yref='y domain',
    xref='paper',
    x=0,
    y=-0.25,
    showarrow=False,
    align='right',
    text='<b>Latency:</b> time elapsed between establishing the TCP connection and receiving the HTTP headers.')
fig

In [8]:
ecdf(df=headers_df, column='download_latency')

## Output

In [9]:
headers_df